In [1]:
import sys
import os
# go to upper diretory
sys.path.append(os.path.abspath('./../../'))
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Audio_Sentiment_Analysis.utils.Configuration import Configuration
from sklearn.preprocessing import minmax_scale
import seaborn as sns
from tabulate import tabulate
import csv

In [2]:
AUDIO_DIR = f"{os.path.abspath('./../../')}/eNTERFACE05_Dataset/*/*/*/*.avi"
CONFIG_FILE = f"{os.path.abspath('./../../')}/Audio_Sentiment_Analysis/data/config.json"
EXTRACTED_FEATURES_FILE = 'extracted_features_ent05.csv'
FEATURES_TABLE_FILE = 'features_table_ent05.csv'
config = Configuration.load_json(CONFIG_FILE)

In [3]:
def load_data(proc_feat_dataset):
    sub_emo_data = defaultdict(lambda: defaultdict(list))

    with open(proc_feat_dataset) as dataset:
        reader = csv.DictReader(dataset)
        for row in reader:
            subject, emotion = row.pop("subject"), row.pop("emotion")
            sub_emo_data[subject][emotion].append(np.array(list(row.values()), dtype=np.float64))

    return sub_emo_data

In [4]:
# Load the processed features writen in the dataset
sub_emo_data = load_data(EXTRACTED_FEATURES_FILE)

In [5]:
def process_features(sub_emo_data):
    data = []

    for subject, sub_data in sub_emo_data.items():
        for emotion, features in sub_data.items():
            # mel_mean mel_min mel_max mel_var mel_std chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate mfcc1 mfcc2...
            features_arr = np.array(features)

            mean_feature_values = [
                int(subject),
                emotion,
                np.mean(features_arr[:,0]),
                np.mean(features_arr[:,1]),
                np.mean(features_arr[:,2]),
                np.mean(features_arr[:,3]),
                np.mean(features_arr[:,4]),
                np.mean(features_arr[:,5]),
                np.mean(features_arr[:,6]),
                np.mean(minmax_scale(features_arr[:,7], config.scale_range)),
                np.mean(features_arr[:,8]),
                np.mean(features_arr[:,9]),
                np.mean(minmax_scale(features_arr[:,10], config.scale_range)),
                np.mean(features_arr[:,11]),
                np.mean(features_arr[:,12]),
                np.mean(features_arr[:,13]),
            ]

            data.append(mean_feature_values)

    return data

In [6]:
# process the extracted features, group and scale them
data = process_features(sub_emo_data)
df = pd.DataFrame(data, columns=["Subject", "Emotion", "mel_mean", "mel_min", "mel_max", "mel_var", "mel_std", "chroma_stft", "rmse", "spectral_centroid", "spectral_bandwidth", "rolloff", "zero_crossing_rate", "mfcc1", "mfcc2", "mfcc3"])
df = df.sort_values(["Subject", "Emotion"], ascending = (True, True))
df = df.set_index('Subject')

df.head()

,Emotion,mel_mean,mel_min,mel_max,mel_var,mel_std,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3
Subject,,,,,,,,,,,,,,,
1,anger,5.448993,3.696495e-10,2593.246057,3196.644043,52.885040,0.351984,0.128167,0.503678,1734.487459,3468.808031,0.569907,-189.488507,119.051169,-38.191239
1,disgust,2.203773,3.467299e-10,1275.522046,629.788232,23.311563,0.396743,0.073791,0.557522,1952.310647,4287.188569,0.552960,-245.959027,100.330852,-27.615436
1,fear,2.062133,3.833569e-10,1171.522095,953.791917,24.186283,0.347003,0.069430,0.440377,1852.591857,3838.981285,0.393695,-261.970770,108.969914,-24.113346
1,happiness,3.596385,4.827979e-10,1385.045239,1395.557080,34.941695,0.373238,0.101833,0.273570,1917.113821,3988.430984,0.269389,-200.898444,107.469267,-30.913523
1,sadness,0.860076,3.661280e-10,618.971106,97.350327,9.813401,0.405850,0.043551,0.537237,1961.970025,4058.874966,0.578430,-302.475494,110.680930,-13.636983


In [7]:
headers = 'subject emotion mel_mean mel_min mel_max mel_var mel_std chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 4):
    headers += f' mfcc{i}'
headers = headers.split()

features_table = tabulate(df, headers=headers, tablefmt="tsv")

with open(FEATURES_TABLE_FILE, "w") as file:
    file.write(features_table)

In [33]:
df = pd.read_csv(FEATURES_TABLE_FILE, index_col="  subject", sep='\t')  
df.head()

,emotion,mel_mean,mel_min,mel_max,mel_var,mel_std,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3
subject,,,,,,,,,,,,,,,
1,anger,5.448990,3.696490e-10,2593.250,3196.6400,52.8850,0.351984,0.128167,0.503678,1734.49,3468.81,0.569907,-189.489,119.051,-38.1912
1,disgust,2.203770,3.467300e-10,1275.520,629.7880,23.3116,0.396743,0.073791,0.557522,1952.31,4287.19,0.552960,-245.959,100.331,-27.6154
1,fear,2.062130,3.833570e-10,1171.520,953.7920,24.1863,0.347003,0.069430,0.440377,1852.59,3838.98,0.393695,-261.971,108.970,-24.1133
1,happiness,3.596390,4.827980e-10,1385.050,1395.5600,34.9417,0.373238,0.101833,0.273570,1917.11,3988.43,0.269389,-200.898,107.469,-30.9135
1,sadness,0.860076,3.661280e-10,618.971,97.3503,9.8134,0.405850,0.043551,0.537237,1961.97,4058.87,0.578430,-302.475,110.681,-13.6370


In [35]:
df.columns = df.columns.str.replace(' ','')
df.groupby(['emotion']).mean()

,mel_mean,mel_min,mel_max,mel_var,mel_std,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3
emotion,,,,,,,,,,,,,,
anger,1.964555,4.979866e-10,784.429767,471.027251,16.464130,0.372403,0.067751,0.488012,1924.475581,3962.216512,0.484149,-278.481140,106.161500,-27.923035
disgust,1.085536,5.411402e-10,484.715302,223.897092,10.423491,0.392498,0.046428,0.478427,2107.384419,4647.360698,0.466020,-311.293744,90.023140,-17.722944
fear,1.103660,5.532540e-10,473.599523,299.937792,10.902051,0.371906,0.044939,0.494855,2096.777674,4493.397442,0.443422,-326.442372,90.857647,-16.201495
happiness,1.675114,5.521772e-10,690.701981,454.379347,15.433534,0.382408,0.060676,0.463871,2018.254884,4374.822558,0.462500,-288.096349,94.076309,-24.391451
sadness,0.438325,5.317131e-10,236.838708,56.091237,4.620724,0.403042,0.026262,0.488531,2148.438372,4585.939535,0.485067,-372.279744,96.234979,-8.827118
surprise,1.179027,5.742984e-10,494.932898,247.988850,11.071982,0.392663,0.047921,0.498313,2020.029535,4202.058372,0.497955,-319.715395,103.760133,-16.978445
